In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2

In [ ]:
import keras
from keras.layers import Dense,Conv2D,Dropout,BatchNormalization,Activation,Flatten,MaxPool2D,Input,LeakyReLU
from keras.models import Sequential
from keras.activations import relu
from keras.optimizers import Adam,RMSprop,SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from sklearn import svm
from keras.preprocessing import image
import xgboost
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as pr

# Visualizing the image

In [ ]:
img = cv2.imread('../input/plant-seedlings-classification/train/Maize/6e9ff31e7.png')
plt.imshow(img)

**Loading the data 128x128 for ANN model**

# Importing Images for Folder for supervise learning model

In [ ]:
train_path='../input/plant-seedlings-classification/train'
test_path='../input/plant-seedlings-classification/test'

In [ ]:
x_train_ann =[];
y_train_ann =[]

for subfolder in os.listdir(train_path):
    for file in os.listdir(train_path+'/' +subfolder):
        path = train_path+'/' +subfolder+ "/" + file
        im = cv2.imread(path)
        ims = cv2.resize(im,(128,128))
        ims = ims/255.0
        x_train_ann.append(ims)
        y_train_ann.append(subfolder)

In [ ]:
le_ann = pr.LabelEncoder()
y_train_le_ann = le_ann.fit_transform(y_train_ann)

x_ann,x_test_ann,y_ann,y_test_ann =  train_test_split(x_train_ann,y_train_le_ann ,test_size=0.2,random_state=10)
#y_train_1_le = le.fit_transform(y_train_1)
x_array_ann = np.array(x_ann)
x_test_arr_ann = np.array(x_test_ann)

In [ ]:
y_train_ann_cat = to_categorical(y_ann)
y_test_ann_cat = to_categorical(y_test_ann)

**Loding Data for Support Vector Machine model 64x64 because of computation limitation**

In [ ]:
x_train = []
y_train = []

In [ ]:
for subfolder in os.listdir(train_path):
    for file in os.listdir(train_path+'/' +subfolder):
        path = train_path+'/' +subfolder+ "/" + file
        im = cv2.imread(path)
        ims = cv2.resize(im,(64,64))
        ims = ims/127.0
        x_train.append(ims)
        y_train.append(subfolder)

In [ ]:
le = pr.LabelEncoder()
y_train_le = le.fit_transform(y_train)


x_svm,x_test_svm,y_svm,y_test_svm =  train_test_split(x_train,y_train_le ,test_size=0.2,random_state=10)
#y_train_1_le = le.fit_transform(y_train_1)
x_array_svm = np.array(x_svm)
x_test_arr_svm = np.array(x_test_svm)

In [ ]:
y_train_cat = to_categorical(y_train_le)
#y_train_1_cat = to_categorical(y_train_1_le)

In [ ]:
flat_arr_size = len(x_array_svm[0])*len(x_array_svm[0][0])*len(x_array_svm[0][0][0])
x_arr_flat_svm = x_array_svm.reshape(len(x_array_svm),flat_arr_size)
x_test_arr_flat_svm = x_test_arr_svm.reshape(len(x_test_arr_svm),flat_arr_size)

# **Model Building - SVM**

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc

In [ ]:
# define support vector classifier
svm = SVC(kernel='linear', probability=True, random_state=10)

# fit model
svm.fit(x_arr_flat_svm, y_svm)

In [ ]:
# generate predictions
y_pred = svm.predict(x_test_arr_flat_svm)

# calculate accuracy
accuracy = accuracy_score(y_test_svm, y_pred)
print('Model accuracy is: ', accuracy)

In [ ]:
import pickle

filename = 'finalized_svm_model.sav'
pickle.dump(svm, open(filename, 'wb'))

# Image importing- for CNN - 256x256

In [ ]:
image_size = 256
batch_size = 32

**To load the images using ImageDataGeneretor class and explicitly indicate the needed parameters (rescaling, flipping, validation splitting.**

In [ ]:
idg = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2
)

**Train set**

In [ ]:
train_gen = idg.flow_from_directory('../input/plant-seedlings-classification/train/',
                                                    target_size=(image_size, image_size),
                                                    subset='training',
                                                    class_mode='categorical',
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    seed=1
                                                )

**Validation set**

In [ ]:
val_gen = idg.flow_from_directory('../input/plant-seedlings-classification/train/',
                                                   target_size=(image_size, image_size),                                                   
                                                   subset='validation',
                                                   class_mode='categorical',
                                                   batch_size=batch_size,
                                                   shuffle=True,
                                                   seed=1
                                                )

In [ ]:
256*256*3

# **2. Model Building - Deep Neural Network**

In [ ]:
model_nn = Sequential()

inputs = Input(shape=(128,128,3))
model_nn.add(tf.keras.layers.Flatten())

model_nn.add(Dense(512,activation = "relu"))
model_nn.add(Dropout(0.2))
# Normalization layer
model_nn.add(tf.keras.layers.BatchNormalization())

model_nn.add(Dense(256,activation = "relu"))
model_nn.add(Dropout(0.2))
# Normalization layer
model_nn.add(tf.keras.layers.BatchNormalization())

model_nn.add(Dense(128,activation = "relu"))
model_nn.add(Dropout(0.2))
# Normalization layer

model_nn.add(tf.keras.layers.BatchNormalization())
model_nn.add(Dense(32,activation = "relu"))
model_nn.add(Dropout(0.2))


model_nn.add(Dense(12,activation='softmax'))
#model_nn.summary()

In [ ]:
model_nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# You can save the best model to the checkpoint
checkpoint_ann = tf.keras.callbacks.ModelCheckpoint('plant_ANN_classifier.h5', #where to save the model
                                                    save_best_only=True, 
                                                    monitor='val_accuracy', 
                                                    mode='max', 
                                                    verbose = 1)

In [ ]:
len(train_gen)
3803//32

In [ ]:
print(len(x_ann[0][0][0]))
print(len(y_train_ann_cat))
y_train_ann_cat[0]

In [ ]:
print(len(x_test_ann))
print(len(y_test_ann_cat))
y_test_ann_cat[0]

Without batch-generator its taking time in training the model- Because of computation limitation just running for 50 Epochs 
with 32 batch sizes

In [ ]:


#model_nn.fit(x_train_ann,y,epochs=150,validation_data=(x_train_ann,y_train_ann_cat),batch_size=128,callbacks = [checkpoint_ann],
 #         verbose = 1)
history = model_nn.fit_generator(train_gen,
          epochs=50, # Increase number of epochs if you have sufficient hardware
          steps_per_epoch= 3803//batch_size,  # Number of train images // batch_size
          validation_data=val_gen,
          validation_steps = 947//batch_size, # Number of val images // batch_size
          callbacks = [checkpoint_ann],
          verbose = 1
)

**Visualize a portion of images to make sure they're correctly loaded.**

In [ ]:
x,y = next(train_gen)

In [ ]:
from mpl_toolkits.axes_grid1 import ImageGrid

def show_grid(image_list, nrows, ncols, label_list=None, show_labels=False, figsize=(10,10)):

    fig = plt.figure(None, figsize,frameon=False)
    grid = ImageGrid(fig, 111, 
                     nrows_ncols=(nrows, ncols),  
                     axes_pad=0.2, 
                     share_all=True,
                     )
    for i in range(nrows*ncols):
        ax = grid[i]
        ax.imshow(image_list[i],cmap='Greys_r')
        ax.axis('off')

In [ ]:
show_grid(x,2,4,show_labels=True,figsize=(10,10))

**The images are nicely loaded and do not have any rotation and distortion.**

# 3. Model building - CNN 

In [ ]:
n_classes = 12;
model = tf.keras.models.Sequential()

# Input layer
# Can be omitted, you can specify the input_shape in other layers
model.add(tf.keras.layers.InputLayer(input_shape=(image_size,image_size,3,)))

# 1. 2D Convolution layer
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))
model.add(tf.keras.layers.BatchNormalization())

# 2. 2D Convolution layer
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), strides = (1,1), activation='relu')) 
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))
model.add(tf.keras.layers.BatchNormalization())

# 3. 2D Convolution layer
model.add(tf.keras.layers.Conv2D(128, kernel_size=(3,3), strides = (1,1), activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))
model.add(tf.keras.layers.BatchNormalization())

# 4. 2D Convolution layer
model.add(tf.keras.layers.Conv2D(128, kernel_size=(3,3), strides = (1,1), activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))
model.add(tf.keras.layers.GlobalMaxPool2D())

# Dense Layers after flattening the data
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(128, activation='relu'))

# Dropout
# is used to nullify the outputs that are very close to zero and thus can cause overfitting.
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())

#Add Output Layer- total 12 classes we have to predict
model.add(tf.keras.layers.Dense(n_classes, activation='softmax')) 

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# You can save the best model to the checkpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint('plant_cnn_classifier.h5', #where to save the model
                                                    save_best_only=True, 
                                                    monitor='val_accuracy', 
                                                    mode='max', 
                                                    verbose = 1)

In [ ]:
history = model.fit(train_gen,
          epochs=30, # Increase number of epochs if you have sufficient hardware
          steps_per_epoch= 3803//batch_size,  # Number of train images // batch_size
          validation_data=val_gen,
          validation_steps = 947//batch_size, # Number of val images // batch_size
          callbacks = [checkpoint],
          verbose = 1
)

**Learning curves vs epoch graph**

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.xticks(list(range(1,50)))
plt.ylim([0, 1])
plt.legend(loc='lower right')

# 4. Conclusion

To classify the image, we have build total 3 models:- 
1. SVM Classifier - Only obtained approx 0.44 accuracy. It can be more but due to computation limitation, i used 64x64 data with default param. No have done any hyperparams tunning, bcz its taking time to get trained

2. Deep Neural Network - Accuracy i got only aprrox .40, we can get more if we will run for more epochs. due to high computation time and resources are required not have tunned the model.

3. CNN Model - Obtained accuracy is approx .93, we can get it more as well but here resouces and time even to achiving 93 its took less time than both DNN and SVN model. Even CNN is also without tunning we are able to reach 93-94 accuracy

Final conclusion is, If image size is smaller then may SVM and DNN can give bit better result but not better than CNN.
Here best model is cnn which save with "plant_cnn_classifier.h5" name

In [ ]:
predict_img = '../input/prediction/Predict.png'
img = cv2.imread(predict_img)
plt.imshow(img)

In [ ]:
data_img = cv2.imread(predict_img)
data_img = cv2.resize(data_img,(256,256))
data_img = data_img/255.0
len(data_img[0])
#model.predict(data_img)

predict_gen = idg.flow_from_directory('../input/prediction',
                                                    target_size=(image_size, image_size),
                                                    subset='training',
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    seed=1
                                                )

*** Convolution Neural Networks or CNN uses filtration to analyze image inputs by casting multiple layers on images. These rectified linear unit layer understand patterns and provide an n-dimensional vector output. To learning param is less so it will easy to train CNN model with higher pixel images. Mainly CNN is to extract the features and it will pass to ANN only to predict but CNN is more capable to create feature due to localization of pixel that its passed to network ***

Task to prepare the data for AI model to predict car manufacturers are popular in a certain area of the city or locality.

There are Mainly Approx 60 Automobile brand. So there are 60 classes we can consider to classify.

If its just image classification then we have to do manual effort to separate car images to there respective classes-
 
As per given image, i can face below difficulties :-

1. We need to collect more data to maintaining the veraities of each car brands and there different product
2. There should be difference combination of images get captured as we can do.
3. Seperate it out all the images to there respestive class lable folders
4. As in given iamges, 2 images have poor pixel quality and in training data we are not able to find it manually that its belong to which brand. So, there should be such example present but correctly placed.
5. Size of images are varying large range. Its should be in smaller range so, after resizing the images it will maintain their features.